In [1]:
# -*- coding: utf-8 -*-
import os
from openai import OpenAI
from pymilvus import model as milvus_model
from pymilvus import MilvusClient
from tqdm import tqdm
import json

In [2]:
# 1. 环境变量与 API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请先设置环境变量 DEEPSEEK_API_KEY")

In [3]:
# 2. 加载民法典文档数据
mfd_path = "../../deepseek/api/mfd.md"
with open(mfd_path, "r", encoding="utf-8") as file:
    mfd_text = file.read()
text_lines = mfd_text.split("# ")

print(f"文档分段数: {len(text_lines)}")
print("示例分段:", text_lines[:5])

文档分段数: 30
示例分段: ['#', '中华人民共和国民法典\n\n##', '（二）物权编\n\n###', '第一章 一般规定\n\n**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。\n\n**第二百零五条** 本编调整因物的归属和利用产生的民事关系。\n\n**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。\n\n**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。\n\n**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n\n**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。\n\n**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。\n\n**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。\n\n**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n\n**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。\n\n**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n\n**第二百一十五条** 不动产登记簿由登记机构管理。\n不动产登记簿应当采用纸质形式或者电子形式。\n不动产登记簿采用电子形式的，应当备份。\n\n*

In [4]:
# 3. 初始化 DeepSeek 客户端
deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",
)

In [5]:
# 4. 初始化 embedding 模型
embedding_model = milvus_model.DefaultEmbeddingFunction()

# 5. 测试 embedding 维度
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print("Embedding 维度:", embedding_dim)
print("Embedding 前10维:", test_embedding[:10])

Embedding 维度: 768
Embedding 前10维: [-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [6]:
# 6. 初始化 Milvus Lite
os.environ["TOKENIZERS_PARALLELISM"] = "false"
milvus_client = MilvusClient(uri="./milvus_mfd.db")
collection_name = "mfd_knowledge_base"

# 7. 清理旧 collection
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

# 8. 创建新 collection
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",
)

/home/dministrator/miniconda3/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [7]:
# 9. 文档嵌入并插入 Milvus
data = []
doc_embeddings = embedding_model.encode_documents(text_lines)
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)
print(f"已插入 {len(data)} 条数据到 Milvus。")

Creating embeddings: 100%|███████████████████████████████████████████████████████| 30/30 [00:00<00:00, 130528.13it/s]

已插入 30 条数据到 Milvus。


In [8]:
# 10. 提出问题并检索
questions = [
    "民法典中关于合同的基本原则是什么？",
    "民法典中如何定义自然人？"
]

for question in questions:
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries([question]),
        limit=3,
        search_params={"metric_type": "IP", "params": {}},
        output_fields=["text"],
    )

    retrieved_lines_with_distances = [
        (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    readable_results = [
        f"段落: {line_with_distance[0]}\n相似度: {line_with_distance[1]:.4f}"
        for line_with_distance in retrieved_lines_with_distances
    ]
    print(f"问题: {question}")
    print("检索结果:")
    print("\n".join(readable_results))

    # 11. 构建 RAG 上下文
    context = "\n".join([line_with_distance[0] for line_with_distance in retrieved_lines_with_distances])

    # 12. 构造提示词
    SYSTEM_PROMPT = """Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。"""
    USER_PROMPT = f"""请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

    # 13. 调用 LLM 生成答案
    response = deepseek_client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ],
    )
    print("RAG 回答：")
    print(response.choices[0].message.content)

问题: 民法典中关于合同的基本原则是什么？
检索结果:
段落: 二、权利质权

**第四百四十九条** 可以出质的权利包括：
（一）汇票、本票、支票；
（二）债券、存款单；
（三）仓单、提单；
（四）可以转让的基金份额、股权；
（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；
（六）应收账款；
（七）法律、行政法规规定可以出质的其他财产权利。

**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。

**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。
以未上市公司股权出质的，适用公司法有关股权转让的规定。

**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。
以未上市公司股权出质的，适用公司法有关股权转让的规定。

**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。

**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。

**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。

**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。

####
相似度: 0.6141
段落: 第三章 合同的变更和转让

**第五百四十八条** 当事人协商一致，可以变更合同。

**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。

**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：
（一）根据合同性质不得转让；
（二）按照当事人约定不得转让；
（三）依照法律规定不得转让。
债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。

**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。

**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经